# Climate Profiles 2

### Step 0: Set-Up
Import the [climakitae](https://github.com/cal-adapt/climakitae) library and other dependencies.

In [1]:
from typing import Tuple

import numpy as np
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm  # Progress bar

import climakitae as ck
from climakitae.explore.standard_year_profile import get_climate_profile, export_profile_to_csv, _get_clean_standardyr_filename, retrieve_profile_data
from climakitae.explore.typical_meteorological_year import TMY
from climakitae.core.data_interface import (
    get_data_options,
    get_subsetting_options,
    get_data,
)

import warnings
warnings.filterwarnings("ignore")

# import functions needed for testing
from climakitae.core.constants import UNSET
from climakitae.core.paths import VARIABLE_DESCRIPTIONS_CSV_PATH
from climakitae.explore.typical_meteorological_year import is_HadISD
from climakitae.explore.standard_year_profile import (
    _get_clean_standardyr_filename,
    _check_stations,
    _check_cached_area,
    _check_lat_lon,
    match_str_to_wl,
)
from climakitae.util.utils import read_csv_file
from climakitae.util.warming_levels import get_gwl_at_year

In [2]:
variable = "Air Temperature at 2m"
units = "degF"
qtile = 0.5
warming_levels = [1.5]
no_delta = True

# warming level window size
valid_window = 5
invalid_window_1 = 2
invalid_window_2 = 5.5

# station name options
station_list = [
    "Sacramento Executive Airport (KSAC)",
    "Santa Barbara Municipal Airport (KSBA)",
]
station_name = ["Sacramento Executive Airport (KSAC)"]
custom_name = ["Custom Station Name"]
mixed_list = [
    "Custom Station Name",
    "Santa Barbara Municipal Airport (KSBA)",
]
custom_list = [
    "Custom Name 1",
    "Custom Name 2",
]

# lat lon
# latitude = 34.4041
# longitude = -121.5160 # no data found for these lat and lon bounds!
latitude = 34.19966
longitude = -118.36543  # no data found for these lat and lon bounds!

# cached area
area_name = "Los Angeles County"

### Time-based file modification

Valid inputs to test
- "approach" = "Time" with a "centered_year" between 2015 and 20199
- no "warming_level" input

Invalid inputs to test
1. "approach" = "Time" with a "centered_year" between 2015 and 2099, with "warming_level" input
2. "approach" = "Time" and no "centered_year" provided
3. "centered_year" outside of 2015-2099
4. "approach" that is not "Time" or "Warming Level"

In [3]:
profile_selections = {
    "variable": variable,
    "resolution": "3 km",
    "q": qtile,
    "warming_level": [1.5],
    "units": units,
    "no_delta": False,
    # # approach
    # "approach": "Time",
    # "centered_year": 2016,
    # # warming level window
    "warming_level_window": 5,
    # # Location options -- uncomment based on your desired location type
    # "stations": station_name,  # uncomment for a weather station
    # "latitude": (
    #     latitude - 0.02,
    #     latitude + 0.02,
    # ),  # uncomment for a using a custom coordinate location
    # "longitude": (
    #     longitude - 0.02,
    #     longitude + 0.02,
    # ),  # uncomment for a custom coordinate location
    "cached_area": area_name, # uncomment for a cached area
}

In [ ]:
profile = get_climate_profile(**profile_selections)

In [ ]:
export_profile_to_csv(profile, **profile_selections)

In [ ]:
name_params = {
    "var_id": "prec",
    "q": 0.75,
    "gwl": None,
    "location": "35-5N_122-5W",
    "no_delta": False,
    "warming_level_window": 5,
    "approach": "Time",
    "centered_year": 2016,
}

In [ ]:
_get_clean_standardyr_filename(**name_params)

### Return only bias-adjusted WRF models

The models are:

    "WRF_EC-Earth3_r1i1p1f1",
    "WRF_MPI-ESM1-2-HR_r3i1p1f1",
    "WRF_TaiESM1_r1i1p1f1",
    "WRF_MIROC6_r1i1p1f1",
    "WRF_EC-Earth3-Veg_r1i1p1f1",

#### Testing

In [4]:
retrieve_selections = {
    "variable": variable,
    "resolution": "45 km",
    # "q": qtile,
    "warming_level": [1.5],
    "units": units,
    # "no_delta": False,
    "bias_adjusted_models": True,
    # # approach
    # "approach": "Time",
    # "centered_year": 2016,
    "time_profile_scenario": "SSP 2-4.5",
    # # warming level window
    "warming_level_window": 5,
    # # Location options -- uncomment based on your desired location type
    # "stations": station_name,  # uncomment for a weather station
    # "latitude": (
    #     latitude - 0.02,
    #     latitude + 0.02,
    # ),  # uncomment for a using a custom coordinate location
    # "longitude": (
    #     longitude - 0.02,
    #     longitude + 0.02,
    # ),  # uncomment for a custom coordinate location
    "cached_area": area_name,  # uncomment for a cached area
}

In [5]:
all_historic, all_future = retrieve_profile_data(**retrieve_selections)

ValueError: No bias-adjusted models available for 'time_profile_scenario' = SSP 2-4.5.

In [5]:
all_historic.coords['simulation'].to_dataframe()

,Lambert_Conformal,centered_year,simulation
simulation,,,
WRF_CESM2_r11i1p1f1_historical+ssp245,0,2019,WRF_CESM2_r11i1p1f1_historical+ssp245
WRF_CESM2_r11i1p1f1_historical+ssp370,0,2016,WRF_CESM2_r11i1p1f1_historical+ssp370
WRF_CESM2_r11i1p1f1_historical+ssp585,0,2016,WRF_CESM2_r11i1p1f1_historical+ssp585
WRF_CNRM-ESM2-1_r1i1p1f2_historical+ssp370,0,2022,WRF_CNRM-ESM2-1_r1i1p1f2_historical+ssp370
WRF_EC-Earth3_r1i1p1f1_historical+ssp370,0,2010,WRF_EC-Earth3_r1i1p1f1_historical+ssp370
WRF_EC-Earth3-Veg_r1i1p1f1_historical+ssp370,0,1995,WRF_EC-Earth3-Veg_r1i1p1f1_historical+ssp370
WRF_FGOALS-g3_r1i1p1f1_historical+ssp370,0,2013,WRF_FGOALS-g3_r1i1p1f1_historical+ssp370
WRF_MIROC6_r1i1p1f1_historical+ssp370,0,2029,WRF_MIROC6_r1i1p1f1_historical+ssp370
WRF_MPI-ESM1-2-HR_r3i1p1f1_historical+ssp370,0,2012,WRF_MPI-ESM1-2-HR_r3i1p1f1_historical+ssp370


All avaialable simulations:

WRF_CESM2_r11i1p1f1_historical+ssp245	
WRF_CESM2_r11i1p1f1_historical+ssp245

WRF_CESM2_r11i1p1f1_historical+ssp585
WRF_CESM2_r11i1p1f1_historical+ssp585


WRF_CESM2_r11i1p1f1_historical+ssp370
WRF_CESM2_r11i1p1f1_historical+ssp370
WRF_CNRM-ESM2-1_r1i1p1f2_historical+ssp370
WRF_CNRM-ESM2-1_r1i1p1f2_historical+ssp370
WRF_EC-Earth3_r1i1p1f1_historical+ssp370
WRF_EC-Earth3_r1i1p1f1_historical+ssp370
WRF_EC-Earth3-Veg_r1i1p1f1_historical+ssp370
WRF_EC-Earth3-Veg_r1i1p1f1_historical+ssp370
WRF_FGOALS-g3_r1i1p1f1_historical+ssp370
WRF_FGOALS-g3_r1i1p1f1_historical+ssp370
WRF_MIROC6_r1i1p1f1_historical+ssp370
WRF_MIROC6_r1i1p1f1_historical+ssp370
WRF_MPI-ESM1-2-HR_r3i1p1f1_historical+ssp370
WRF_MPI-ESM1-2-HR_r3i1p1f1_historical+ssp370
WRF_TaiESM1_r1i1p1f1_historical+ssp370	

bias-adjusted simulations:

